# Train a neural network for binary classification

In this notebook, we will use Nobrainer to train a model to classify brains into one of two classes. This task is different from segmentation -- the model will output a single scalar per brain.

In the following cells, we will:

1. Get sample T1-weighted MR scans as features and random boolean values as labels.
2. Convert the data to TFRecords format.
3. Create two Datasets of the features and labels.
    - One dataset will be for training and the other will be for evaluation.
4. Instantiate a 3D convolutional neural network.
5. Choose a loss function and metrics to use.
6. Train on part of the data.
7. Evaluate on the rest of the data.

## Google Colaboratory

If you are using Colab, please switch your runtime to GPU. To do this, select `Runtime > Change runtime type` in the top menu. Then select GPU under `Hardware accelerator`. A GPU greatly speeds up training.

In [ ]:
import nobrainer

# Get sample features and labels

We use 9 pairs of volumes for training and 1 pair of volumes for evaulation. Many more volumes would be required to train a model for any useful purpose.

In [ ]:
import random

csv_of_filepaths = nobrainer.utils.get_data()
filepaths = nobrainer.io.read_csv(csv_of_filepaths)

# Add random boolean values
filepaths = [(x, random.choice([0, 1])) for x, _ in filepaths]

train_paths = filepaths[:9]
evaluate_paths = filepaths[9:]

In [ ]:
filepaths

# Convert medical images to TFRecords

Remember how many full volumes are in the TFRecords files. This will be necessary to know how many steps are in on training epoch. The default training method needs to know this number, because Datasets don't always know how many items they contain.

In [ ]:
# Verify that all volumes have the same shape and that labels are integer-ish.

invalid = nobrainer.io.verify_features_labels(train_paths)
assert not invalid

invalid = nobrainer.io.verify_features_labels(evaluate_paths)
assert not invalid

In [ ]:
!mkdir -p data

In [ ]:
# Convert training and evaluation data to TFRecords.

nobrainer.tfrecord.write(
    features_labels=train_paths,
    filename_template='data/data-train_shard-{shard:03d}.tfrec',
    examples_per_shard=3)

nobrainer.tfrecord.write(
    features_labels=evaluate_paths,
    filename_template='data/data-evaluate_shard-{shard:03d}.tfrec',
    examples_per_shard=1)

In [ ]:
!ls data

# Create Datasets

In [ ]:
n_classes = 1
batch_size = 2
volume_shape = (256, 256, 256)
block_shape = (128, 128, 128)
n_epochs = None
num_parallel_calls = 2

In [ ]:
dataset_train = nobrainer.dataset.get_dataset(
    file_pattern="data/data-train_shard*.tfrec",
    n_classes=n_classes,
    batch_size=batch_size,
    volume_shape=volume_shape,
    scalar_label=True,
    block_shape=block_shape,
    n_epochs=n_epochs,
    num_parallel_calls=num_parallel_calls,
)

dataset_evaluate = nobrainer.dataset.get_dataset(
    file_pattern="data/data-evaluate_shard-*.tfrec",
    n_classes=n_classes,
    batch_size=batch_size,
    volume_shape=volume_shape,
    scalar_label=True,
    block_shape=block_shape,
    n_epochs=1,
    num_parallel_calls=num_parallel_calls,
)

# Instantiate a neural network

In [ ]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Conv3D(filters=12, kernel_size=3, padding="same", 
                           strides=2, activation="relu", input_shape=(*block_shape, 1)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128),
    tf.keras.layers.Dense(n_classes, activation="sigmoid")
])

In [ ]:
model.summary()

# Choose a loss function and metrics

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-04)

model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.binary_crossentropy,
    metrics=["accuracy"],
)

# Train and evaluate model

$$
steps = \frac{nBlocks}{volume} * \frac{nVolumes}{batchSize}
$$

In [ ]:
steps_per_epoch = nobrainer.dataset.get_steps_per_epoch(
    n_volumes=len(train_paths),
    volume_shape=volume_shape,
    block_shape=block_shape,
    batch_size=batch_size)

steps_per_epoch

In [ ]:
validation_steps = nobrainer.dataset.get_steps_per_epoch(
    n_volumes=len(evaluate_paths),
    volume_shape=volume_shape,
    block_shape=block_shape,
    batch_size=batch_size)

validation_steps

In [ ]:
model.fit(
    dataset_train,
    epochs=5,
    steps_per_epoch=steps_per_epoch, 
    validation_data=dataset_evaluate, 
    validation_steps=validation_steps)

# Predict natively medical images (without TFRecords)

In [ ]:
# TODO